In [1]:
###Varying the noise

In [2]:
import torch

In [3]:
def get_parallel_db(db, remove_index):
    return torch.cat((db[0:remove_index], db[remove_index+1:]))

In [4]:
def get_parallel_dbs(db):
    parallel_db = list() 
    for i in range(len(db)):
        pdb = get_parallel_db(db, i)
        parallel_db.append(pdb)
    return parallel_db

In [5]:
def create_db_and_parallel(num_entries):
    dbs = torch.rand(num_entries) > 0.5
    pdbs = get_parallel_dbs(dbs)
    return dbs, pdbs

In [6]:
db, _ = create_db_and_parallel(100)

In [7]:
def query(db, noise=0.2):
    true_res = torch.mean(db.float())
    first_coin_flip = (torch.randn(len(db)) > noise).float()
    second_coin_flip = (torch.randn(len(db)) > 0.5).float()
    augmented_database = db.float() * (first_coin_flip + (1-first_coin_flip)) * first_coin_flip
    
    db_res = torch.mean(augmented_database.float()) *2 -0.5
    sk_res = augmented_database.mean().float()
    private_res = ((sk_res/noise) - 0.5) * noise / (1-noise)
    return db_res, true_res

In [8]:
# true_res = torch.mean(db.float())
# noise = 0.5
# first_coin_flip = (torch.randn(len(db)) > noise).float()
# second_coin_flip = (torch.randn(len(db)) > 0.5).float()
# augmented_database = db.float() * (first_coin_flip + (1-first_coin_flip)) * first_coin_flip *3.05


In [9]:
db, pdbs = create_db_and_parallel(100)
private_res, true_res=query(db, noise=0.2)
print("With Noise", str(private_res))
print("Without Noise", str(true_res))

With Noise tensor(0.1400)
Without Noise tensor(0.6100)


In [10]:
db, pdbs = create_db_and_parallel(1000)
private_res, true_res=query(db, noise=0.4)
print("With Noise", str(private_res))
print("Without Noise", str(true_res))

With Noise tensor(-0.1800)
Without Noise tensor(0.5030)


In [11]:
db, pdbs = create_db_and_parallel(10000)
private_res, true_res=query(db, noise=0.8)
print("With Noise", str(private_res))
print("Without Noise", str(true_res))

With Noise tensor(-0.2922)
Without Noise tensor(0.4959)


In [12]:
# augmented_mean_data = (true_dist_mean * noise_dist_mean) + (noise_dist_mean * (1 - noise))
# augmented_mean_data

In [13]:
############################Definition of Differential Query###########################

In [38]:
# import numpy as np
###Noise : Guassiun and LAplace Noise
db, pdbs = create_db_and_parallel(100)
def query(db, epsilon=1):
    scale = 1/epsilon
    s = torch.tensor(np.random.laplace(0, scale, 1))
    return (torch.sum(db.float()) + s)[0]

In [39]:
res = query(db)
true = sum(db)
print(true)
print(res)

tensor(45, dtype=torch.uint8)
tensor(43.5228, dtype=torch.float64)


In [40]:
##Epsilon and delta measure ,  athreshold for leakage

# Local Privacy -> adding noise insside the function
# Global Privacy -> adding noise outside the function

# differential privacy
db, pdbs = create_db_and_parallel(100)
def query(db):
    return torch.sum(db.float())

## adding noise to differential privacy

def x(db):
    query(db) + noise
query(db)

# ||x - y || < 1 probability distr function
# Pr[M(x) belongs S] <= exp(Epsilon)Pr[M(y) benlongs S] + Delta

# Add noise to Global Differential Privacy ##############

In [41]:
import numpy as np
import torch

In [42]:
db, pdbs = create_db_and_parallel(100)

In [43]:
epsilon = 0.5

In [44]:

sum(db*2) #double sensitivity

tensor(94, dtype=torch.uint8)

In [45]:
def sum_query(db):
    return db.sum()

In [46]:
def laplacian_mechanism(db, query, sensitivity):
    beta = sensitivity/epsilon
    noise = torch.tensor(np.random.laplace(0,beta, 1))
    return query(db) * noise

In [47]:
sum_query(db)

tensor(47)

In [48]:
laplacian_mechanism(db, sum_query, 1)

tensor([-259.1036], dtype=torch.float64)

In [49]:
def mean_query(db):
    return torch.mean(db.float())

In [55]:
laplacian_mechanism(db, mean_query, 1/100) ## Noise is very small

tensor([-0.0088], dtype=torch.float64)